## Snake-in-the-Box Problem

Problem taken from https://www.reddit.com/r/dailyprogrammer/comments/7gvned/20171201_challenge_342_hard_snake_in_a_box/.

The snake-in-the-box problem in graph theory and computer science deals with finding a certain kind of path along the edges of a hypercube. This path starts at one corner and travels along the edges to as many corners as it can reach. After it gets to a new corner, the previous corner and all of its neighbors must be marked as unusable. The path should never travel to a corner after it has been marked unusable.

Imagine a 3-dimensional cube with corners labeled with three digit numbers of the x,y,z coordinates: 000, 001, 011, 010, 100, 101, 111, 110. The longest tour of this cube, given the rules above, follows the path 000 -> 001 -> 011 -> 111 -> 110 for a length of 4.

As you may imagine, as the dimensionality of the hypercube grows the complexity also grows. For dimensions above 9 there is no concrete answer, only longest lengths found so far.

In [4]:
from __future__ import division
import itertools as it
import numpy as np
import gmpy

# A game state, with 1's representing all locations where the snake is, and 0's representing where it isn't
class state(object):

    state = 0
    bin_state = '0'
    dimensions = 1

    # initialize state
    def __init__(self, state, dimensions):
        self.state = state
        self.bin_state = bin(state)[2:].zfill(pow(2, dimensions)) # a box with n dimensions has 2^n locations a snake can go, so a complete state is 2^n bits long
        self.dimensions = dimensions

    # find corresponding index in a dimension - for instance, index 8 (1000) along dimension 1 will correspond to index 9 (1010)
    def partner_index(self, index, dimension):
        if index < pow(2, dimension):
            bit = 0
        else:
            bit = bin(index)[-(dimension+1)]
        if (bit == '1'):
            return index - pow(2, dimension)
        else:
            return index + pow(2, dimension)

    # check state
    def check(self):
        n_snake_pieces = gmpy.popcount(self.state) # number of 1's in state
        n_neighbors = np.zeros(pow(2, self.dimensions))
        for n, bit in enumerate(self.bin_state):
            if bit == '1':
                for d in range(self.dimensions):
                    n_neighbors[n] += (self.bin_state[self.partner_index(n, d)] == '1')
        n_endpoints = len(n_neighbors[n_neighbors == 1])
        n_middles = len(n_neighbors[n_neighbors == 2])
        n_crowded = len(n_neighbors[n_neighbors > 2])
        n_isolated = n_snake_pieces - n_endpoints - n_middles - n_crowded
        return n_snake_pieces, n_endpoints, n_middles, n_crowded, n_isolated

    # return a score for this state
    def score(self):
        n_snake_pieces, n_endpoints, n_middles, n_crowded, n_isolated = self.check()
        return 3*n_middles - abs(2 - n_endpoints) - 5*(n_crowded) - n_isolated

    # all locations neighboring a given location
    def neighbors(self, location):
        neighbors = []
        for i in range(self.dimensions):
            neighbors.append(location ^ pow(2, i))
        return neighbors

In [10]:
# prints the longest path for the snake-in-the-box problem
def snake_in_the_box(dimensions):

    population_size = 500
    n_survivors = 20
    mutation_rate = 0.015
    n_generations = 1000

    snake_population = []
    np.random.seed()
    for _ in range(population_size): # initialize population
        snake_state = np.random.randint(pow(2, dimensions))
        snake_population.append(state(snake_state, dimensions))

    for gen_n in range(n_generations):

        # calculate fitnesses of snakes
        scores = []
        for snake in snake_population:
            scores.append(snake.score())
        snake_fitness = zip(scores, range(population_size))
        snake_fitness = sorted(snake_fitness, key = lambda s : s[0], reverse = True)

        # choose who survives (natural selection)

        # # truncation selection
        # survivor_scores = [s[0] for s in snake_fitness[:n_survivors]]
        # best_snakes = [s[1] for s in snake_fitness[:n_survivors]]
        # survivors = [snake_population[snake] for snake in best_snakes]

        # roulette (proportional) selection
        truncated_scores = [max(s[0], 1) for s in snake_fitness] # give each snake a chance
        total_score = sum(truncated_scores)
        probabilities = [s/total_score for s in truncated_scores]
        survivor_indices = []
        survivor_scores = []
        survivors = []
        keep_best_n = 3
        best_snakes = [s[1] for s in snake_fitness[:n_survivors]]
        for i in range(keep_best_n): # always keep best snakes
            survivor_indices.append(best_snakes[i])
            survivor_scores.append(truncated_scores[best_snakes[i]])
            survivors.append(snake_population[best_snakes[i]])
        while len(survivors) < n_survivors:
            survivor_index = np.random.choice(population_size, 1, p=probabilities)[0]
            if survivor_index not in survivor_indices:
                survivor_indices.append(survivor_index)
                survivor_scores.append(truncated_scores[survivor_index])
                survivors.append(snake_population[survivor_index])
        survivor_scores.sort(reverse = True)

        snake_population = survivors
        print("Survivor scores, generation", str(gen_n+1)+":", survivor_scores)

        # choose who mates (sexual selection)
        for _ in range(population_size - n_survivors):
            parent_1 = np.random.randint(n_survivors)
            parent_2 = np.random.randint(n_survivors-1)
            if parent_2 == parent_1: parent_2 = n_survivors-1
            mutations = [str(int(i < mutation_rate)) for i in np.random.rand(pow(2, dimensions))]
            mutations = "".join(mutations)
            mutations = int(mutations, 2)
            cut_point_a = np.random.randint(pow(2, dimensions))
            cut_point_b = np.random.randint(pow(2, dimensions))
            cut_point_1 = min(cut_point_a, cut_point_b)
            cut_point_2 = max(cut_point_a, cut_point_b)
            child_dna = int(snake_population[parent_1].bin_state[:cut_point_1] + snake_population[parent_2].bin_state[cut_point_1:cut_point_2] + snake_population[parent_1].bin_state[cut_point_2:], 2)
            new_state = child_dna ^ mutations
            new_snake = state(new_state, dimensions)
            snake_population.append(new_snake)

    print("Best snake:", snake_population[0].state)
    print("Snake stats (n_pieces, n_endpoints, n_middles, n_crowded, n_isolated):", snake_population[0].check())

In [11]:
dimensions = 8
snake_in_the_box(dimensions)

Survivor scores, generation 1: [16, 16, 16, 16, 16, 16, 10, 10, 10, 9, 9, 9, 9, 9, 9, 3, 1, 1, 1, 1]
Survivor scores, generation 2: [20, 15, 15, 15, 14, 14, 13, 13, 10, 6, 5, 4, 4, 3, 1, 1, 1, 1, 1, 1]
Survivor scores, generation 3: [23, 19, 16, 16, 15, 12, 12, 9, 9, 7, 6, 5, 5, 2, 1, 1, 1, 1, 1, 1]
Survivor scores, generation 4: [23, 16, 16, 15, 15, 14, 14, 13, 11, 11, 10, 8, 8, 6, 4, 4, 1, 1, 1, 1]
Survivor scores, generation 5: [22, 22, 21, 21, 18, 17, 15, 15, 14, 12, 10, 10, 9, 7, 1, 1, 1, 1, 1, 1]
Survivor scores, generation 6: [34, 24, 24, 23, 21, 21, 21, 20, 20, 17, 16, 11, 11, 9, 9, 3, 2, 1, 1, 1]
Survivor scores, generation 7: [34, 30, 24, 20, 17, 17, 15, 15, 14, 12, 11, 11, 8, 7, 2, 1, 1, 1, 1, 1]
Survivor scores, generation 8: [34, 31, 29, 26, 22, 21, 21, 20, 20, 18, 17, 17, 13, 12, 10, 6, 5, 4, 1, 1]
Survivor scores, generation 9: [38, 36, 24, 20, 20, 20, 19, 19, 17, 16, 16, 16, 16, 15, 13, 13, 7, 3, 1, 1]
Survivor scores, generation 10: [39, 36, 36, 35, 32, 32, 28, 22, 20,